In [1]:
!pip install transformers==4.40.1 accelerate==0.30.0 bitsandbytes==0.43.1 datasets==2.19.0 vllm==0.4.1 openai==1.25.1 -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 810.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.6/222.6 MB 6.1 MB/s eta 0:00:00
   ━━

# **실습에 사용할 데이터셋 준비**

In [2]:
import torch
from datasets import load_dataset

def make_prompt(ddl, question, query=''):
  prompt = f"""당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.

### DDL:
{ddl}

### Question:
{question}

### SQL:
{query}"""

  return prompt

dataset = load_dataset("shangrilar/ko_text2sql", "origin")['test']
dataset = dataset.to_pandas()

for idx, row in dataset.iterrows():
  prompt = make_prompt(row['context'], row['question'])
  dataset.loc[idx, 'prompt'] = prompt

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/38246 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/112 [00:00<?, ? examples/s]

# **모델과 토크나이저를 불러와 추론 파이프라인 준비**

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "shangrilar/yi-ko-6b-text2sql"
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_id)

hf_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/9.74k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.28M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# **배치 크기에 따른 추론 시간 확인**

In [4]:
import time
for batch_size in [1, 2, 4, 8, 16, 32]:
  start_time = time.time()
  # max_new_tokens -> 최대 생성할 토큰 개수
  hf_pipeline(dataset['prompt'].tolist(), max_new_tokens=128, batch_size=batch_size)
  print(f'{batch_size}: {time.time() - start_time}')

1: 274.06331515312195
2: 416.80681109428406
4: 291.19162583351135


KeyboardInterrupt: 

# **vLLM 모델 불러오기**

In [ ]:
import torch
from vllm import LLM, SamplingParams

model_id = "shangrilar/yi-ko-6b-text2sql"
# max_model_len=1024 -> 모델이 한 번에 처리할 최대 토큰 수를 1024로 제한
llm = LLM(model=model_id, dtype=torch.float16, max_model_len=1024)

# **vLLM을 활용한 오프라인 추론 시간 측정**

In [ ]:
import time

for max_num_seqs in [1, 2, 4, 8, 16, 32]:
  start_time = time.time()
  # 한 번에 처리할 수 있는 시퀀스 개수를 조절
  # 더 많은 요청을 병렬로 처리
  llm.llm_engine.scheduler_config.max_num_seqs = max_num_seqs

  # 샘플링 파라미터
  # 주로 텍스트 생성 모델에서 생성되는 텍스트의 특성을 제어하는 설정값
  # 모델이 텍스트를 어떻게 선택할지 결정하는 중요한 요소

  # 샘플링 파라미터는 텍스트의 다양성, 예측 가능성, 창의성 등을 조정할 수 있도록 도와준다

  # Temperature (온도):
  # 이 값은 모델이 생성할 단어를 선택할 때 얼마나 무작위성을 추가할지를 결정한다
  # 낮은 값 (0 ~ 0.3)은 더 예측 가능한 텍스트를 생성하게 하고, 높은 값 (0.7 ~ 1은 더 창의적이고 다양성 있는 텍스트를 생성한다
  # 예를 들어 temperature=1일 경우 모델이 다양한 단어를 선택하려는 경향이 더 커지며, temperature=0.2일 경우 모델은 더 예측 가능한 단어를 선택할 가능성이 크다.

  # Top-p (누적 확률):
  # top_p는 "nucleus sampling"이라는 샘플링 기법과 관련이 있다. 모델은 확률이 높은 상위 일부 단어들만 고려하여 텍스트를 생성한다.
  # top_p=1일 경우 모든 단어를 고려하고, top_p=0.9라면 확률의 누적 합이 90%에 해당하는 단어들만 선택된다.
  # 이렇게 하면 확률 분포가 넓게 퍼지지 않아서 텍스트 생성이 더 일관되게 된다.

  # Top-k (상위 k개 후보):
  # 이 방법은 모델이 생성할 때 상위 k개의 후보 단어들만 고려하여 샘플링한다.
  # 예를 들어 top_k=50이면 가장 확률이 높은 50개의 단어만 선택하고 그 중 하나를 랜덤하게 선택한다.
  sampling_params = SamplingParams(temperature=1, top_p=1, max_tokens=128)
  outputs = llm.generate(dataset['prompt'].tolist(), sampling_params)

  print(f'{max_num_seqs}: {time.time() - start_time}')